In [1]:
from datasets import load_dataset

ds = load_dataset("microsoft/ms_marco", "v1.1")

# Collecting and splitting datasets


In [2]:
train_dataset = load_dataset("microsoft/ms_marco", "v1.1", split="train")
valid_dataset = load_dataset("microsoft/ms_marco", "v1.1", split="validation")
test_dataset  = load_dataset("microsoft/ms_marco", "v1.1", split="test")

In [3]:
import pandas as pd
main = pd.DataFrame(train_dataset)
main.head()

,answers,passages,query,query_id,query_type,wellFormedAnswers
0,[Results-Based Accountability is a disciplined...,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",what is rba,19699,description,[]
1,[Yes],"{'is_selected': [0, 1, 0, 0, 0, 0, 0], 'passag...",was ronald reagan a democrat,19700,description,[]
2,[20-25 minutes],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",how long do you need for sydney and surroundin...,19701,numeric,[]
3,[$11 to $22 per square foot],"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 1], '...",price to install tile in shower,19702,numeric,[]
4,[Due to symptoms in the body],"{'is_selected': [0, 0, 1, 0, 0, 0, 0, 0], 'pas...",why conversion observed in body,19703,description,[]


# Create Query db


In [4]:
query_db = pd.DataFrame()
query_db = main[['query','query_id']]
query_db.head()

,query,query_id
0,what is rba,19699
1,was ronald reagan a democrat,19700
2,how long do you need for sydney and surroundin...,19701
3,price to install tile in shower,19702
4,why conversion observed in body,19703


# Create Document db


In [5]:
docs = []
for x in range(0,len(main['passages'])):
    for doc in main['passages'][x]['passage_text']:
        docs.append(doc)

print('doc_count: ', len(docs))
#remove duplicates
docs = set(docs)
print('\nde-deuplicated docs: ', len(docs))

doc_count:  676193

de-deuplicated docs:  626907


In [6]:
docs = list(enumerate(docs))
print(docs[:5])

[(0, 'The central nervous system relies on pathways and higher order functions to transfer information to various parts of the brain, giving the body sensory and motor ability. There is a constant interaction going on between the brain, spinal cord, and the peripheral nervous system (PNS). Pathways are either ascending (sensory) or descending (motor) and contain tracts and nuclei, where signals from one neuron to another creates a synapse. They are in the skin, joints, muscles and viscera (Noback 155). Sensory systems transmit information regarding intensity. Receptors of pain are called nociceptors. Each of the pathways involve a chain of neurons named accordingly, a first order neuron is a neuron that delivers directly to the CNS or its cell body.'), (1, 'The NGC contains 7,840 objects, known as the NGC objects. It is one of the largest comprehensive catalogues, as it includes all types of deep space objects and is not confined to, for example, galaxies. Dreyer published two suppleme

In [7]:
doc_db = pd.DataFrame()
doc_db = pd.DataFrame([{'document_id': i[0], 'document': i[1]} for i in docs])
doc_db.head()

,document_id,document
0,0,The central nervous system relies on pathways ...
1,1,"The NGC contains 7,840 objects, known as the N..."
2,2,Escitalopram is not associated with significan...
3,3,1 Venus is the hottest planet in the Solar Sys...
4,4,Let’s look at the numbers Investing outside of...


In [ ]:
###An abomination of a program I wrote to do what I acheived in the cell below. Not my best work 
# query_doc_rows = []
# positive_document = ""
# for x in range(0, len(main['passages'])):
#     for idx, select in enumerate(main['passages'][x]['is_selected']):
#         if select: # wher iter == 1 . [0, 0, 1, 0, 0] and [0, 1, 1, 0, 0] types exist
#             positive_document = main['passages'][x]['passage_text'][idx]
#             match = doc_db.loc[doc_db['document'] == positive_document, 'document_id']

#             if not match.empty:
#                 positive_document_id = match.iloc[0]
#                 # print("\npositive_doc_id, ", positive_document_id)
#                 # print("query_id, ", main['query_id'][x])
#                 query_doc_rows.append({
#                 'query_id': main['query_id'][x],
#                 'positive_doc_id': positive_document_id
#                 })

KeyboardInterrupt: 

In [9]:
positive_rows = []

for x in range(len(main['passages'])):
    for idx, select in enumerate(main['passages'][x]['is_selected']):
        if select == 1:
            positive_rows.append({
                'query_id': main['query_id'][x],
                'document': main['passages'][x]['passage_text'][idx]
            })

positive_df = pd.DataFrame(positive_rows)
query_doc_db = positive_df.merge(doc_db, on='document', how='inner')
query_doc_db = query_doc_db[['query_id', 'document_id']].rename(columns={'document_id': 'positive_doc_id'})

In [10]:
query_doc_db = pd.DataFrame(query_doc_rows)
query_doc_db.head()

,query_id,positive_doc_id
0,19699,269519
1,19700,436823
2,19701,111815
3,19702,318722
4,19703,593515


In [11]:
## Export Dataframes to csv
query_db.to_csv("./data/query_db.csv", index=False)
doc_db.to_csv("./data/doc_db.csv", index=False)
query_doc_db.to_csv("./data/query_doc_db.csv", index=False)